In [8]:
import tifffile
import functools
import json
import os
import re
from typing import Mapping, Tuple, Union

import click
import numpy as np
from slicedimage import ImageFormat

import starfish.core.util.try_import
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, Features, Number

ROUND_NUM = 1
FOV_NUM = 1

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return tifffile.imread(file_path)

class SampleDataTile(FetchedTile):

    def __init__(self, 
                 file_path: str, 
                 coordinates: Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]],
                 z: int) -> None:
        """
        Parameters
        ----------
        file_path : str
            location of the tile
        coordinates : Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]]
            the coordinates for the selected tile, extracted from the metadata
        z : int
            the z-layer for the selected osmFISH tile
        """
        self.file_path = file_path
        self._coordinates = coordinates
        self.z = z
        
    @property
    def shape(self) -> Mapping[Axes, int]:
        raw_shape = self.tile_data().shape
        return {Axes.Y: raw_shape[0], Axes.X: raw_shape[1]}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], Union[Number, Tuple[Number, Number]]]:
        return self._coordinates

    def tile_data(self) -> np.ndarray:
        return tifffile.imread(self.file_path)[self.z] # slice out the correct z-plane

class SampleDataTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        basename = f"round01-ch00.tif" # assume the first file of the series is named this way
        first_file = os.path.join(input_dir, basename)
        
        raw_shape = cached_read_fn(first_file).shape
        self.num_c = raw_shape[3] if len(raw_shape) > 3 else 1
        self.num_z = raw_shape[0]
        
    @property
    def channel_map(self) -> Mapping[str, int]:
        if self.num_c >1:
            return {
                "red": 0,
                "blue": 1,
                "green": 2,
            }
        return 0
    
    def coordinate_map(self, round_: int, z: int):
        # TODO see osmFISH for example on how to dynamically generate this
        
        # dummy coordinates
        return {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
    
    def get_tile(self, fov: int, r: int, ch: int, z: int) -> FetchedTile:
        basename = f"round0{r + 1}-ch0{ch}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = self.coordinate_map(r, z)
        return SampleDataTile(file_path, coordinates, z)
    
#     def generate_codebook(self, output_dir: str) -> None:
#         """
#         TODO
#         Generate and save a codebook from the provided mapping of genes to DNA sequences.
#         output_dir : str
#             directory in which to save the generated codebook. Codebook is saved as "codebook.json"

#         """
#         dinucleotides_to_channels = {
#             "CT": 3,
#             "GT": 2,
#             "TT": 1,
#             "AG": 3,
#             "GG": 1,
#             "TG": 2,
#             "AC": 2,
#             "CC": 1,
#             "TC": 3,
#             "AA": 1,
#             "CA": 2,
#             "GA": 3,
#         }
        
#         with open(os.path.join(self.input_dir, "genes.csv"), "r") as f:
#             codes = [l.strip().split(",") for l in f.readlines()]  # List[(gene, dna_barcode), ...]

#         def iter_dinucleotides(sequence):
#             i = 0
#             while i + 1 < len(sequence):
#                 yield sequence[i:i + 2]
#                 i += 1

#         # construct codebook target mappings
#         code_array = []
#         for gene, dna_barcode in codes:
#             dna_barcode = dna_barcode[::-1]  # reverse barcode
#             spacetx_barcode = [
#                 {
#                     Axes.ROUND.value: r,
#                     Axes.CH.value: dinucleotides_to_channels[dinucleotide],
#                     Features.CODE_VALUE: 1
#                 } for r, dinucleotide in enumerate(iter_dinucleotides(dna_barcode))
#             ]
#             code_array.append({
#                 Features.CODEWORD: spacetx_barcode,
#                 Features.TARGET: gene
#             })

#         codebook = Codebook.from_code_array(code_array)
#         codebook.to_json(os.path.join(output_dir, "codebook.json"))

# class SampleDataDapiTileFetcher(TileFetcher):

#     def __init__(self, input_dir: str) -> None:
#         """
#         The red channel in the TIF files contains staining that labels all cells, 
#         which can serve a similar function to DAPI
#         """
#         self.input_dir = input_dir

#     def get_tile(self, fov: int, r: int, ch: int, z: int) -> FetchedTile:
#         basename = f"round0{r + 1}-ch03.tif"
#         file_path = os.path.join(self.input_dir, basename)
#         return StarMapTile(file_path, z)

    
def cli(input_dir, output_dir) -> None:
    """
    TODO
    osmFISH example does not use auxillary images - could try, but need dimensions
    """
    abs_output_dir = os.path.expanduser(output_dir)
    abs_input_dir = os.path.expanduser(input_dir)
    os.makedirs(abs_output_dir, exist_ok=True)

    primary_tile_fetcher = SampleDataTileFetcher(abs_input_dir)
    primary_image_dimensions: Mapping[Union[str, Axes], int] = {
        Axes.ROUND: ROUND_NUM,
        Axes.CH: primary_tile_fetcher.num_c,
        Axes.ZPLANE: primary_tile_fetcher.num_z,
    }

    write_experiment_json(
        path=output_dir,
        fov_count=FOV_NUM,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        primary_tile_fetcher=primary_tile_fetcher,
        aux_name_to_dimensions={},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )

    #primary_tile_fetcher.generate_codebook(abs_output_dir)
    
if __name__ == "__main__":
    cli('sample_data/w1_second_raw','sample_data/w1_second_formatted')

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x12ceec2b0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a72f830> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a729bf8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a73efc0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x12ced8410> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108e00ba0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108df96d0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108dfc200> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108e1dd00> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108e23830> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108e122b0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108dfc678> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108e110a0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108e20ba0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x108e2a6d0> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a72e0a0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a74bf10> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a734308> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a753d00> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a7b2d58> is a low contrast image
  warn('%s is a low 

/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x12ceac9e8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/Users/danyfu/.pyenv/versions/3.6.3/lib/python3.6/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x22a75feb8> is a low contrast image
  warn('%s is a low contrast image' % fname)


In [ ]:
#smFISH example does not have cell segmentation step
# only filter, spot detection, and decode

#Segmentation used in in-situ sequencing (ISS) example
# 1) detect 2) decode 3) segment cell 4) assign decoded spots to cells

In [47]:
from typing import Optional, Tuple
from IPython import get_ipython

import starfish
import starfish.data
from starfish import FieldOfView, IntensityTable

In [31]:
experiment = starfish.Experiment.from_json("sample_data/w1_second_formatted/experiment.json")
for key in experiment.keys():
    print(key)

fov_000


In [35]:
from starfish.types import Axes
fov0 = experiment['fov_000']

In [36]:
# ipython = get_ipython()
# ipython.magic("gui qt5")

# primary_images = fov0.get_image(FieldOfView.PRIMARY_IMAGES)
# raw_image_viewer = starfish.display(image.sel({Axes.CH: 0, Axes.ROUND: 0}))
# raw_image_viewer

100%|██████████| 169/169 [00:11<00:00, 14.37it/s]


In [40]:
images = enumerate(fov0.get_images(FieldOfView.PRIMARY_IMAGES))
for image_number, primary_image in enumerate(images):
    print(image_number)

100%|██████████| 169/169 [00:08<00:00, 19.13it/s]

0


In [ ]:
# bandpass filter to remove cellular background and camera noise
bandpass = starfish.image.Filter.Bandpass(lshort=.5, llong=7, threshold=0.0)
# gaussian blur to smooth z-axis
glp = starfish.image.Filter.GaussianLowPass(
    sigma=(1, 0, 0),
    is_volume=True
)

# pre-filter clip to remove low-intensity background signal
clip1 = starfish.image.Filter.Clip(p_min=50, p_max=100)
# post-filter clip to eliminate all but the highest-intensity peaks
clip2 = starfish.image.Filter.Clip(p_min=99, p_max=100, is_volume=True)

# spot detection
tlmpf = starfish.spots.DetectSpots.TrackpyLocalMaxPeakFinder(
    spot_diameter=5,  # must be odd integer
    min_mass=0.02,
    max_size=2,  # this is max radius
    separation=7,
    noise_size=0.65,  # this is not used because preprocess is False
    preprocess=False,
    percentile=10,  # this is irrelevant when min_mass, spot_diameter, and max_size are set properly
    verbose=True,
    is_volume=True,
)

for image_number, primary_image in enumerate(fov0.get_images(FieldOfView.PRIMARY_IMAGES)):
    print(f"Filtering image {image_number}...")
    filter_kwargs = dict(
        in_place=False,
        verbose=True,
        n_processes=None
    )
    clipped1 = clip1.run(primary_image, **filter_kwargs)
    clipped1.to_multipage_tiff('sample_data/w1_second_filtered/first_clip_w1_fov000.tiff')
    
    bandpassed = bandpass.run(clipped1, **filter_kwargs)
    bandpassed.to_multipage_tiff('sample_data/w1_second_filtered/bandpassed_w1_fov000.tiff')
    
    glped = glp.run(bandpassed, **filter_kwargs)
    glped.to_multipage_tiff('sample_data/w1_second_filtered/lowpass_filtered_w1_fov000.tiff')
    
    clipped2 = clip2.run(glped, **filter_kwargs)
    clipped2.to_multipage_tiff('sample_data/w1_second_filtered/final_filtered_w1_fov000.tiff')
    

In [62]:
print("Calling spots...")
spot_attributes = tlmpf.run(clipped2)
print(spot_attributes)

Calling spots...
<xarray.IntensityTable (features: 17076, c: 1, r: 1)>
array([[[0.008359]],

       [[0.003768]],

       ...,

       [[0.001766]],

       [[0.001766]]])
Coordinates:
    eccentricity     (features) float64 nan nan nan nan nan ... nan nan nan nan
    ep               (features) float64 nan nan nan nan nan ... nan nan nan nan
    radius           (features) float64 1.401 1.421 1.43 ... 1.439 1.443 1.46
    raw_mass         (features) float64 0.05602 0.02765 ... 0.03189 0.03415
    total_intensity  (features) float64 0.05322 0.02496 ... 0.02967 0.03226
    x                (features) float64 218.0 467.0 ... 1.077e+03 1.038e+03
    y                (features) float64 1.309e+03 1.41e+03 ... 1.979e+03
    z                (features) float64 1.991 7.0 12.01 ... 166.0 166.1 166.1
  * features         (features) int64 0 1 2 3 4 ... 17072 17073 17074 17075
  * c                (c) int64 0
  * r                (r) int64 0
    xc               (features) float64 1.065e-05 2.281e

In [65]:
spot_attributes.to_netcdf('w1_second_intensity_table.nc')

In [68]:
spot_attributes[20][0][0]

<xarray.IntensityTable ()>
array(0.003532)
Coordinates:
    eccentricity     float64 nan
    ep               float64 nan
    radius           float64 1.427
    raw_mass         float64 0.06235
    total_intensity  float64 0.06028
    x                float64 756.3
    y                float64 1.321e+03
    z                float64 29.06
    features         int64 20
    c                int64 0
    r                int64 0
    xc               float64 3.695e-05
    yc               float64 6.454e-05
    zc               float64 5e-05
Attributes:
    starfish:  {"log": [{"method": "Clip", "arguments": {"p_min": 50, "p_max"...